# Logistic Regression Example

In this tutorial, we'll walk through how to use our automatic differentiation engine to create and train a logistic regression model to predict heart disease. 

The data can be found [here](https://www.kaggle.com/saxarchit/logistic-regression).

Our goal here is not to create the best model to fit our data, but to simply show how to use our automatic differentiation engine to create, train, and test a logistic regression model. Thus, our model will be very simplistic for pedagogical purposes.

# Data Preprocessing

We'll first take look at our data, validate it, and preprocess it to be in a form appropriate to use with our automatic differentiation engine.

Feel free to skip this section as it doesn't directly relate to how to use our automatic differentiation engine.

Let's first import some necessary libraries.

In [1]:
from autograd import Variable, LinearLayer
import autograd
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

Let's load the data. 

In [2]:
csv_path = './data/framingham.csv'
df = pd.read_csv(csv_path)
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


Let's clean our data.

Since this tutorial is focused on using our automatic differentiaion engine rather than on machine learning itself, we'll avoid doing a deep dive into the data analysis rationalizing the methods we here to clean our data. More examples of analysis on this dataset can be found [here](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset/notebooks).

Let's first drop any null values.

In [3]:
df.dropna(axis=0, inplace=True)
assert df.isnull().sum().sum() == 0

Our model will be very simple. We'll create a logistic regression model that takes in a value from each column and will return whether or not the patient has a 10 year risk of heart disease.

Our automatic differentiation engine expects numpy arrays. 

Let's convert the data into two numpy arrays, one for the inputs and one for the outputs.

In [4]:
input_df = df[['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
output_df = df['TenYearCHD']
input_matrix = input_df.values
output_vector = output_df.values

In [5]:
type(input_matrix)

numpy.ndarray

In [6]:
input_matrix.shape

(3658, 15)

In [7]:
type(output_vector)

numpy.ndarray

In [8]:
output_vector.shape

(3658,)

Let's now split our data into a training set and test set. We'll use 70% of our data for training and 30% for testing. We'll manually set the random seeds for reproducibility purposes.

In [9]:
random.seed(0)
np.random.seed(0)

row_count = input_matrix.shape[0]
row_indices = list(range(row_count))
random.shuffle(row_indices)

last_training_row_position = round(row_count*0.75)
training_indices = row_indices[:last_training_row_position+1]
testing_indices = row_indices[last_training_row_position+1:]

training_input_matrix = input_matrix[training_indices]
training_output_vector = output_vector[training_indices]
testing_input_matrix = input_matrix[testing_indices]
testing_output_vector = output_vector[testing_indices]

A logistic regression model can be implemented as a dense linear layer with a sigmoid activation. 

This is how we'll implement our model. 

In [10]:
sigmoid = lambda x: np.exp(x)/(np.exp(x)+1)
linear_layer = LinearLayer(15, 1)

First, we'll need to establish some hyperparameters for training our linear regression model. We'll be using stochastic gradient descent.

In practice, we might use various batch sizes or shuffle our training data between epochs, but we'll forgo this for the sake of simplicity as the purposes of this tutorial is to show how to use our automatic differentiation engine rather than finding the best model possible.

In [32]:
number_of_epochs = 1
learning_rate = 1e-4
sgd = autograd.optimizer.SGD(learning_rate)

Let's write our training loop. 

In [33]:
linear_layer.matrix *= 0
linear_layer.biases *= 0

epoch_iterator = tqdm(range(number_of_epochs))
for epoch_index in epoch_iterator:
    for x, y in zip(training_input_matrix, training_output_vector):
        linear_result = linear_layer(x)
        y_hat = sigmoid(linear_result)
        diff = (y_hat - y)
        loss = diff ** 2
        var2grad = sgd.take_training_step(loss)
    epoch_iterator.set_description(f'Loss: {loss.sum().data:.8f}')
    epoch_iterator.refresh()

Loss: 0.04586753: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


Let's now see how our model performs on our test set.

In [ ]:
mean_testing_loss = 0
for x, y in zip(testing_input_matrix, testing_output_vector):
    y_hat = np.sum(np.multiply(y, weights) + biases)
    loss = (y_hat - y) ** 2
    mean_testing_loss += loss.data
mean_testing_loss /= len(testing_output_vector)

In [ ]:
mean_testing_loss

Let's look at a few random results to sanity check our mode. 

In [ ]:
random_test_examples = random.sample(list(zip(testing_input_matrix, testing_output_vector)), 10)
for x, y in random_test_examples:
    y_hat = np.sum(np.multiply(y, weights) + biases)
    print()
    print(f'y_hat: {y_hat.data}')
    print(f'y:     {y}')